Test action space sample

In [2]:
from sb3_contrib import MaskablePPO
import edge_relocation as er
# create environment
erEnv = er.EdgeRelEnv("topoconfig.json")
erEnv.action_space.sample()

14

Test masks

In [4]:
erEnv.reset()
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for mec in erEnv.mec_nodes:
    print(mec.id, "cpu util: ", mec.cpu_utilization, "mem util: ", mec.memory_utilization)
print(erEnv.action_masks())

#please note that masks refers to the valid action of selecting 'mec1', mec2, mec3 etc, while the list below lists mecs in wrong order

App reqlatency:  25 current MEC ID:  mec15 current cell ID:  31
mec8 cpu util:  52 mem util:  52
mec9 cpu util:  74 mem util:  74
mec10 cpu util:  41 mem util:  41
mec7 cpu util:  23 mem util:  23
mec12 cpu util:  24 mem util:  24
mec13 cpu util:  39 mem util:  39
mec14 cpu util:  54 mem util:  54
mec11 cpu util:  79 mem util:  79
mec16 cpu util:  82 mem util:  82
mec17 cpu util:  67 mem util:  67
mec18 cpu util:  23 mem util:  23
mec15 cpu util:  52 mem util:  62
mec20 cpu util:  59 mem util:  59
mec21 cpu util:  42 mem util:  42
mec22 cpu util:  44 mem util:  44
mec19 cpu util:  51 mem util:  51
mec2 cpu util:  13 mem util:  13
mec3 cpu util:  67 mem util:  67
mec4 cpu util:  46 mem util:  46
mec5 cpu util:  82 mem util:  82
mec6 cpu util:  58 mem util:  58
mec1 cpu util:  65 mem util:  65
[True, True, True, True, False, True, True, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True]


Test Maskable PPO

In [1]:
import edge_relocation as er

import datetime


# create environment
erEnv = er.EdgeRelEnv("topoconfig.json")
obs = erEnv.reset()
log_dir = "../logs/PPO/" + datetime.datetime.now().strftime("MultiEnVRewNormDelay%Y%m%d-%H%M%S")
model = MaskablePPO("MultiInputPolicy", erEnv, verbose=1, tensorboard_log = log_dir)
model.learn()

#Seems that for today, MaskablePPO is checking about the type of Space, and it accepts only spaces of gym, not yet a spaces of gymnasium. Further checks required

NameError: name 'MaskablePPO' is not defined